In [ ]:
import gc 
gc.collect()

# Morlet Time Frequency Analysis - Visualizations

### Modules

In [ ]:
import mne
import pandas as pd
import numpy as np
import pickle
import os

from matplotlib import pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns

import math
from scipy import stats
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import fdrcorrection

### Define dictionaries, subject, & conditions

In [ ]:
# path to data files
#data_path = ".../data/"
data_path = "/Users/annalorenz/Documents/Research/INS_Paper/data_pub"

# list of conditions
condition_list = ['produce_music', 'perceive_music_produced', 'produce_speech', 'perceive_speech_produced']
contrasts = [["produce_music", "perceive_music_produced"], ["produce_speech", "perceive_speech_produced"]]
contrast_names = ["Music Production and Music Perception", "Speech Production and Speech Perception"]

condition_list_control = ['perceive_music_new', 'perceive_music_newrepetition', 'perceive_speech_new', 'perceive_speech_newrepetition']
contrasts_control = [["perceive_music_new", "perceive_music_newrepetition"], ["perceive_speech_new", "perceive_speech_newrepetition"]]
contrast_names_control = ["Music Perception Control and Music Perception Control Repetition", "Speech Perception Control and Speech Perception Control Repetition"]

### Load epoch data 

In [ ]:
# load epochs
epochs = {}

for condition in condition_list:
    preprocessed_path = data_path + "/preprocessed/" + condition + "/"
    
    for files in os.listdir(preprocessed_path):
        filename = "day1_bipolar_epochs_preprocessed.fif"

        if filename in files:
            path = preprocessed_path + files + '/'
            epochs[condition] = mne.read_epochs(path)   

In [ ]:
# channel names
picks_H = epochs["produce_music"].ch_names
print(picks_H)

# parameters
tmin = 0 
tmax = 300 #2.999
fmin = 1
fmax = 180
frequencies = np.logspace(np.log10(fmin), np.log10(fmax), num = 50, base = 10)  # log10 freq scale 


### Retrieve TFA results

In [ ]:
day = 'both'

In [ ]:
morlet_results = {}
morlet_results_day1 = {}
morlet_results_day2 = {}

for condition in condition_list:
    morlet_results[condition] = {}
    morlet_results_day1[condition] = {}
    morlet_results_day2[condition] = {}
      
    concatenated_results = {}

    file_path_day1 = f"{data_path}/analysis/TFA/{condition}_day1_morlet_results.pickle"
    with open(file_path_day1, 'rb') as f:
        day1 = pickle.load(f)

    file_path_day2 = f"{data_path}/analysis/TFA/{condition}_day2_morlet_results.pickle"
    with open(file_path_day2, 'rb') as f:
        day2 = pickle.load(f)

    morlet_results[condition] = np.concatenate((day1, day2))
    morlet_results_day1[condition] = day1
    morlet_results_day2[condition] = day2

In [ ]:
morlet_results_control = {}

for condition in condition_list_control:
    morlet_results_control[condition] = {}
        
    file_path_control = f"{data_path}/analysis/TFA/{condition}_day1_morlet_results.pickle"
    with open(file_path_control, 'rb') as f:
        control = pickle.load(f)

    morlet_results_control[condition] = control

# Compare conditions

In [ ]:
freq_band_index = [[0,13], [13,20], [20,24], [24,33], [33,42], [42,50]]
band_names = ["Delta", "Theta", "Alpha", "Beta", "Low Gamma", "High Gamma"]

## Self Produced Speech and Music: Production - Perception 

In [ ]:
contrasts = [["produce_music", "perceive_music_produced"], ["produce_speech", "perceive_speech_produced"]]
channel_indices = np.r_[42:50, 89:96]
print(picks_H)
exponent = 1

for n, contrast in enumerate(contrasts): 

    if "music" in contrast[0]: 
        color_prod = 'purple'
        color_perc = 'plum'
    elif "speech" in contrast[0]: 
        color_prod = '#008000'
        color_perc = '#90EE90'

    for channel_index, pick in zip(channel_indices, picks_H): 

        # Calculate mean across epochs
        data_produced = np.mean(morlet_results[contrast[0]][:, channel_index, :], axis=0) 
        data_new = np.mean(morlet_results[contrast[1]][:, channel_index, :], axis=0)

        # Calculate standard error of the mean (SEM)
        sem_produced = np.std(morlet_results[contrast[0]][:, channel_index, :], axis=0)  / np.sqrt(morlet_results[contrast[0]].shape[0])
        sem_new = np.std(morlet_results[contrast[1]][:, channel_index, :], axis=0) / np.sqrt(morlet_results[contrast[1]].shape[0])

        # Mitigate the 1/f effect by multiplying power values by frequency vector
        data_produced *= frequencies ** exponent
        data_new *= frequencies ** exponent
        sem_produced *= frequencies ** exponent
        sem_new *= frequencies ** exponent

        # Plot the Power Spectrum Density with SEM
        fig, ax = plt.subplots(figsize=(12, 3), ncols=1)

        # Plot the mean power with SEM shading for production
        ax.plot(np.arange(len(frequencies)), data_produced, color=color_prod, label='Production', linewidth=2)
        ax.fill_between(np.arange(len(frequencies)), data_produced - sem_produced, data_produced + sem_produced, color=color_prod, alpha=0.3)

        # Plot the mean power with SEM shading for perception
        ax.plot(np.arange(len(frequencies)), data_new, color=color_perc, label='Perception', linewidth=2)
        ax.fill_between(np.arange(len(frequencies)), data_new - sem_new, data_new + sem_new, color=color_perc, alpha=0.3)

        ax.set_ylabel('Power', fontsize=16)
        ax.text(-0.05, 1.20, f'{pick}', fontsize=16, transform=ax.transAxes, ha='left', va='top')
        ax.legend(fontsize=12, loc='upper right')

        ax.set_xticks(np.arange(len(frequencies)))
        ax.set_xticklabels(np.around(frequencies, 1), rotation=90, size=12)
        ax.set_xlim(-0.5, len(frequencies) - 0.5)

        # Adding vertical lines to separate frequency bands
        for band_index in [[0, 13], [13, 20], [20, 24], [24, 33], [33, 42]]:
            ax.axvline(band_index[1] - 0.5, color="black", lw=1)

        # Adding a secondary x-axis for frequency bands
        ax2 = ax.twiny()  
        ax2.set_xlim(ax.get_xlim()) 
        band_midpoints = [(index[0] + index[1]) / 2 for index in freq_band_index]
        ax2.set_xticks(band_midpoints)  
        ax2.set_xticklabels(band_names, rotation=0, ha='center', fontsize=12)  
        ax2.tick_params(axis='x', pad=10)  
        fig.subplots_adjust(bottom=0.2, top=0.85)

        plt.show()


In [ ]:
for n, contrast in enumerate(contrasts): 
    if "music" in contrast[0]: 
        color_prod = 'purple'
        color_perc = 'plum'
    elif "speech" in contrast[0]: 
        color_prod = '#008000'
        color_perc = '#90EE90'

    num_channels = len(channel_indices)
    ncol = 3
    nrow = math.ceil(num_channels / ncol)

    fig, axes = plt.subplots(nrow, ncol, figsize=(12, 3 * nrow), sharex=True, sharey=False)
    axes = axes.flatten()

    for idx, (channel_index, pick) in enumerate(zip(channel_indices, picks_H)):
        ax = axes[idx] 

        # Calculate mean and SEM
        data_produced = np.mean(morlet_results[contrast[0]][:, channel_index, :], axis=0) 
        data_new = np.mean(morlet_results[contrast[1]][:, channel_index, :], axis=0)
        sem_produced = np.std(morlet_results[contrast[0]][:, channel_index, :], axis=0) / np.sqrt(morlet_results[contrast[0]].shape[0])
        sem_new = np.std(morlet_results[contrast[1]][:, channel_index, :], axis=0) / np.sqrt(morlet_results[contrast[1]].shape[0])

        # Mitigate 1/f effect
        data_produced *= frequencies ** exponent
        data_new *= frequencies ** exponent
        sem_produced *= frequencies ** exponent
        sem_new *= frequencies ** exponent

        ax.plot(np.arange(len(frequencies)), data_produced, color=color_prod, label='Production', linewidth=2)
        ax.fill_between(np.arange(len(frequencies)), data_produced - sem_produced, data_produced + sem_produced, color=color_prod, alpha=0.3)
        ax.plot(np.arange(len(frequencies)), data_new, color=color_perc, label='Perception', linewidth=2)
        ax.fill_between(np.arange(len(frequencies)), data_new - sem_new, data_new + sem_new, color=color_perc, alpha=0.3)
        #ax.text(-0.05, 1.20, f'{pick}', fontsize=16, transform=ax.transAxes, ha='left', va='top')
        ax.text(0.35, 1.20, f'{pick}', fontsize=16, transform=ax.transAxes, ha='left', va='top')

        xticks = np.arange(0, len(frequencies), 4) 
        xtick_labels = np.around(frequencies[::4], 1) 

        ax.set_xticks(xticks)
        ax.set_xticklabels(xtick_labels, rotation=90, size=8)
        ax.set_xlim(-0.5, len(frequencies) - 0.5)

        for band_index in freq_band_index:
            ax.axvline(band_index[1] - 0.5, color="black", lw=1)
        
        ax.legend(fontsize=8, loc='upper right')


    fig.subplots_adjust(bottom=0.1, top=0.95, hspace=0.4)
    plt.show()


In [ ]:
channel_index = np.r_[42:50, 89:96]
brain_label = ["Right Primary \n Auditory Cortex", "Right Associative \n Auditory Cortex", "Left Primary \n Auditory Cortex", "Left Associative \n Auditory Cortex"]
channel_position = [[0,5], [6,7], [8,11], [11,len(picks_H)]]

for n, contrast in enumerate(contrasts): 
    data_produced = morlet_results[contrast[0]][:,channel_index,:]
    data_new = morlet_results[contrast[1]][:,channel_index,:]
    
    ratio = np.mean(((data_produced - data_new ) / data_new * 100), axis=0)

    t, p = stats.ttest_rel(data_produced, data_new)
    p_corrected = multipletests(p.flatten(), method='fdr_bh')[1].reshape(p.shape)
    t[p_corrected > 0.05] = 0
    
    fig, ax = plt.subplots(figsize=(12, 8), ncols=1)

    im1 = ax.imshow(t, aspect='auto', origin='lower', vmin=-10, vmax=10, interpolation='none',  cmap='seismic')
    #im1 = ax.imshow(ratio, aspect='auto', origin='lower', interpolation='none', vmax=80, vmin=-80, cmap="seismic")
   
    ax.set_xlabel('Frequencies', size=16)
    ax.set_ylabel('Channels', size=16)
    ax.set_yticks(np.arange(t.shape[0]))
    ax.set_yticklabels(picks_H, size=12)
    ax.set_xticks(np.arange(len(frequencies)))
    ax.set_xticklabels((np.around(frequencies,1)), rotation=90, size=12)
    ax.axhline(7.5, color="black", lw=3)
    for channel_pos_index in [[0,5], [5,7], [7,11]]: 
        ax.axhline(channel_pos_index[1] + 0.5, color="black", lw=1.5)
    for band_index in [[0,13], [13,20], [20,24], [24,33], [33,42]]:
        ax.axvline(band_index[1] - 0.5, color="black", lw=1)

    ax2 = ax.twiny()  
    ax2.set_xlim(ax.get_xlim()) 
    band_midpoints = [(index[0] + index[1]) / 2 for index in freq_band_index]
    ax2.set_xticks(band_midpoints)  
    ax2.set_xticklabels(band_names, rotation=0, ha='center', size=12)  
    ax2.tick_params(axis='x', pad=10)  
    fig.subplots_adjust(bottom=0.2, top=0.85)

    ax3 = ax.twinx()  
    ax3.set_ylim(ax.get_ylim()) 
    channel_midpoints = [(index[0] + index[1]) / 2 for index in channel_position]
    ax3.set_yticks(channel_midpoints)  
    ax3.set_yticklabels(brain_label, size=12, ha='left')  

    colorbar = fig.colorbar(im1, ax=ax, fraction=0.05, pad=0.17)
    colorbar.set_label('t-statistics', size=14) 

    contrast_name = contrast[0] + "-" + contrast[1]
    #plt.savefig(plot_path + f"TFA_H-channels_t-statistics_{contrast_name}_power-frequency-plot.jpg", bbox_inches='tight')
    plt.show()


In [ ]:
brain_label = ["Right Primary \n Auditory Cortex", "Right Associative \n Auditory Cortex", "Left Primary \n Auditory Cortex", "Left Associative \n Auditory Cortex"]
channel_position = [[0,5], [6,7], [8,11], [11,len(picks_H)]]
channel_index = np.r_[42:50, 89:96]

for i, morlet_results in enumerate([morlet_results_day1, morlet_results_day2]):
    for n, contrast in enumerate(contrasts): 
        data_produced = morlet_results[contrast[0]][:,channel_index,:]
        data_new = morlet_results[contrast[1]][:,channel_index,:]
        
        t, p = stats.ttest_rel(data_produced, data_new)
        p_corrected = multipletests(p.flatten(), method='fdr_bh')[1].reshape(p.shape)
        t[p_corrected > 0.05] = 0
        
        fig, ax = plt.subplots(figsize=(12, 8), ncols=1)

        im1 = ax.imshow(t, aspect='auto', origin='lower', vmin=-10, vmax=10, interpolation='none',  cmap='seismic')
    
        ax.set_xlabel('Frequencies', size=16)
        ax.set_ylabel('Channels', size=16)
        ax.set_yticks(np.arange(t.shape[0]))
        ax.set_yticklabels(picks_H, size=12)
        ax.set_xticks(np.arange(len(frequencies)))
        ax.set_xticklabels((np.around(frequencies,1)), rotation=90, size=12)
        ax.axhline(7.5, color="black", lw=3)
        for channel_pos_index in [[0,5], [5,7], [7,11]]: 
            ax.axhline(channel_pos_index[1] + 0.5, color="black", lw=1.5)
        for band_index in [[0,13], [13,20], [20,24], [24,33], [33,42]]:
            ax.axvline(band_index[1] - 0.5, color="black", lw=1)

        ax2 = ax.twiny()  
        ax2.set_xlim(ax.get_xlim()) 
        band_midpoints = [(index[0] + index[1]) / 2 for index in freq_band_index]
        ax2.set_xticks(band_midpoints)  
        ax2.set_xticklabels(band_names, rotation=0, ha='center', size=12)  
        ax2.tick_params(axis='x', pad=10)  
        fig.subplots_adjust(bottom=0.2, top=0.85)

        ax3 = ax.twinx()  
        ax3.set_ylim(ax.get_ylim()) 
        channel_midpoints = [(index[0] + index[1]) / 2 for index in channel_position]
        ax3.set_yticks(channel_midpoints)  
        ax3.set_yticklabels(brain_label, size=12, ha='left')  

        colorbar = fig.colorbar(im1, ax=ax, fraction=0.05, pad=0.17)
        colorbar.set_label('t-statistics', size=14) 

        contrast_name = contrast[0] + "-" + contrast[1]
        #plt.savefig(plot_path + f"TFA_H-channels_t-statistics_{contrast_name}_{day}_power-frequency-plot.jpg", bbox_inches='tight')
        plt.show()


## Linear mixed effect models


In [ ]:
def custom_round(val, threshold=1e-5, decimals=3):
    if abs(val) < threshold:
        return f"{val:.{decimals}e}"
    else:
        return round(val, decimals)

In [ ]:
# Define channel groups
picks_primary_RH = ['H5-H6', 'H6-H7', 'H7-H8', 'H8-H9', 'H9-H10', 'H10-H11']
picks_primary_LH = ["H'5-H'6", "H'6-H'7", "H'7-H'8", "H'8-H'9"]
picks_nonprimary_RH = ['H11-H12', 'H12-H13']
picks_nonprimary_LH = ["H'9-H'10", "H'10-H'11", "H'11-H'12"]

# Create a dictionary for mapping channels to regions
channel_to_region = {}

# Add each channel group to the mapping
for ch in picks_primary_RH:
    channel_to_region[ch] = 'Primary_RH'
for ch in picks_primary_LH:
    channel_to_region[ch] = 'Primary_LH'
for ch in picks_nonprimary_RH:
    channel_to_region[ch] = 'Non_primary_RH'
for ch in picks_nonprimary_LH:
    channel_to_region[ch] = 'Non_primary_LH'

### Model for music

In [ ]:
channel_index = np.r_[42:50, 89:96]
fmin = 1
fmax = 180
frequencies = np.logspace(np.log10(fmin), np.log10(fmax), num = 50, base = 10)  # log10 freq scale 
epochs=np.r_[0:100]
data = []
for day, morlet_results in zip(['Day1', 'Day2'], [morlet_results_day1, morlet_results_day2]):
    for condition in ['produce_music', 'perceive_music_produced']:
        for pick, channel_idx in zip(picks_H, channel_index):
            for freq_idx, freq in enumerate(frequencies):
                for epoch_idx in epochs:
                    result = morlet_results[condition][epoch_idx, channel_idx, freq_idx]
                    data.append({
                        'Condition': condition, 
                        'Day': day, 
                        'Channel': pick, 
                        'Frequency': freq, 
                        'Epoch': epoch_idx, 
                        'Score': result
                    })

data_music = pd.DataFrame(data)

data_music['Condition'] = pd.Categorical(data_music['Condition'], 
                                          categories=['produce_music', 'perceive_music_produced'],
                                          ordered=True)

data_music['Region'] = data_music['Channel'].map(channel_to_region)


In [ ]:
freq_band_index = [[0, 13], [13, 20], [20, 24], [24, 33], [33, 42], [42, 49]]
band_names = ["Delta", "Theta", "Alpha", "Beta", "Low Gamma", "High Gamma"]
freq_band_ranges = [(frequencies[fmin], frequencies[fmax]) for fmin, fmax in freq_band_index]

model_results = {}

# Loop over frequency bands
for (fmin, fmax), band_name in zip(freq_band_ranges, band_names):
    if band_name == "High Gamma":
        data_band = data_music[(data_music['Frequency'] >= fmin) & (data_music['Frequency'] <= fmax)]
    else:
        data_band = data_music[(data_music['Frequency'] >= fmin) & (data_music['Frequency'] < fmax)]
    
   # print(data_band.head())
    print(band_name)
    print(f"shape: {data_band.shape}")
    print("column names:", data_band.columns)
    unique_scores_count = data_band['Score'].nunique()
    print(f"\nNumber of unique scores in the band: {unique_scores_count}")

    unique_frequencies_count = data_band['Frequency'].unique()
    print(f"Unique frequencies in the band: {unique_frequencies_count}")
    print("\n")

In [ ]:
# Compare Model
freq_band_index = [[0, 13], [13, 20], [20, 24], [24, 33], [33, 42], [42, 49]]
band_names = ["Delta", "Theta", "Alpha", "Beta", "Low Gamma", "High Gamma"]
freq_band_ranges = [(frequencies[fmin], frequencies[fmax]) for fmin, fmax in freq_band_index]


model_results = {}
results_list = []

# Loop over frequency bands
for (fmin, fmax), band_name in zip(freq_band_ranges, band_names):
    if band_name == "High Gamma":
        data_band = data_music[(data_music['Frequency'] >= fmin) & (data_music['Frequency'] <= fmax)]
    else:
        data_band = data_music[(data_music['Frequency'] >= fmin) & (data_music['Frequency'] < fmax)]
    
    data_band_avg = data_band.groupby(['Day', 'Condition', 'Channel', 'Epoch'], as_index=False).agg({
        'Score': 'mean'  #
    })
    
    data_band_avg['Region'] = data_band_avg['Channel'].map(channel_to_region)

    # Model 1: Fixed effect of Condition 
    model1 = smf.mixedlm(
        formula="Score ~ C(Condition)",  
        data=data_band_avg,
        groups=data_band_avg["Channel"]  
    ).fit(reml=False)
    model_results[band_name] = {"Model Condition": model1}


    # Model 2: Fixed effects of Condition and Region
    model2 = smf.mixedlm(
        formula="Score ~ C(Condition) + C(Region)",  
        data=data_band_avg,
        groups=data_band_avg["Channel"]  
    ).fit(reml=False)

    model_results[band_name]["Model Condition + Region"] = model2

    # Likelihood Ratio Test statistic
    ll_model_1 = model1.llf  
    ll_model_2 = model2.llf  
    num_params_model_1 = len(model1.params)
    num_params_model_2 = len(model2.params)
    
    lr_statistic = 2 * (ll_model_2 - ll_model_1)
    df_diff = num_params_model_2 - num_params_model_1
    p_value = 1 - stats.chi2.cdf(lr_statistic, df_diff)

    print(f"\n{band_name}: Likelihood Ratio Test between Model 1 and Model 2: Statistic = {lr_statistic}, p-value = {p_value}")

    results_list.append({
        "Stimuli": "music", 
        "Frequency Band": band_name,
        "Model1": "Condition",
        "Model2": "Condition + Region",
        "AIC_Model1": model1.aic,
        "BIC_Model1": model1.bic,
        "AIC_Model2": model2.aic,
        "BIC_Model2": model2.bic,
        "Log-Likelihood_Model1": ll_model_1,
        "Log-Likelihood_Model1": ll_model_2,
        "LR Statistic": lr_statistic,
        "p-value": p_value
    })

df_results = pd.DataFrame(results_list)

In [ ]:
# Compare Model
freq_band_index = [[0, 13], [13, 20], [20, 24], [24, 33], [33, 42], [42, 49]]
band_names = ["Delta", "Theta", "Alpha", "Beta", "Low Gamma", "High Gamma"]
freq_band_ranges = [(frequencies[fmin], frequencies[fmax]) for fmin, fmax in freq_band_index]

model_results = {}
results_list = []

# Loop over frequency bands
for (fmin, fmax), band_name in zip(freq_band_ranges, band_names):
    if band_name == "High Gamma":
        data_band = data_music[(data_music['Frequency'] >= fmin) & (data_music['Frequency'] <= fmax)]
    else:
        data_band = data_music[(data_music['Frequency'] >= fmin) & (data_music['Frequency'] < fmax)]
    
    data_band_avg = data_band.groupby(['Day', 'Condition', 'Channel', 'Epoch'], as_index=False).agg({
        'Score': 'mean'  #
    })
    
    data_band_avg['Region'] = data_band_avg['Channel'].map(channel_to_region)

    # Model 1: Fixed effect of Condition + Region
    model1 = smf.mixedlm(
        formula="Score ~ C(Condition) + C(Region)",  
        data=data_band_avg,
        groups=data_band_avg["Channel"]  
    ).fit(reml=False)
    model_results[band_name] = {"Model Condition + Region": model1}


    # Model 2: Fixed effects of Condition * Region
    model2 = smf.mixedlm(
        formula="Score ~ C(Condition) * C(Region)",  
        data=data_band_avg,
        groups=data_band_avg["Channel"]  
    ).fit(reml=False)

    model_results[band_name]["Model Condition * Region"] = model2

    # Likelihood Ratio Test statistic
    ll_model_1 = model1.llf  
    ll_model_2 = model2.llf  
    num_params_model_1 = len(model1.params)
    num_params_model_2 = len(model2.params)
    
    lr_statistic = 2 * (ll_model_2 - ll_model_1)
    df_diff = num_params_model_2 - num_params_model_1
    p_value = 1 - stats.chi2.cdf(lr_statistic, df_diff)

    print(f"\n{band_name}: Likelihood Ratio Test between Model 1 and Model 2: Statistic = {lr_statistic}, p-value = {p_value}")

    results_list.append({
        "Stimuli": "music", 
        "Frequency Band": band_name,
        "Model1": "Condition + Region",
        "Model2": "Condition * Region",
        "AIC_Model1": model1.aic,
        "BIC_Model1": model1.bic,
        "AIC_Model2": model2.aic,
        "BIC_Model2": model2.bic,
        "Log-Likelihood_Model1": ll_model_1,
        "Log-Likelihood_Model1": ll_model_2,
        "LR Statistic": lr_statistic,
        "p-value": p_value
    })


df_results = pd.DataFrame(results_list)

In [ ]:
# Model "Score ~ C(Condition)"
freq_band_index = [[0, 13], [13, 20], [20, 24], [24, 33], [33, 42], [42, 49]]
band_names = ["Delta", "Theta", "Alpha", "Beta", "Low Gamma", "High Gamma"]
freq_band_ranges = [(frequencies[fmin], frequencies[fmax]) for fmin, fmax in freq_band_index]

model_results = {band_name: {} for band_name in band_names}

# Loop over frequency bands
for (fmin, fmax), band_name in zip(freq_band_ranges, band_names):
    if band_name == "High Gamma":
        data_band = data_music[(data_music['Frequency'] >= fmin) & (data_music['Frequency'] <= fmax)]
    else:
        data_band = data_music[(data_music['Frequency'] >= fmin) & (data_music['Frequency'] < fmax)]
    
    data_band_avg = data_band.groupby(['Day', 'Condition', 'Channel', 'Epoch'], as_index=False).agg({
        'Score': 'mean'  
    })
    
    data_band_avg['Region'] = data_band_avg['Channel'].map(channel_to_region).fillna('Unknown') 

    # Model 
    model_condition = smf.mixedlm(
        formula="Score ~ C(Condition)", 
        data=data_band_avg,                            
        groups=data_band_avg["Channel"]     
    ).fit(reml=False)
    model_results[band_name] = {"Model Condition": model_condition}

results_list = []

# Loop through the regression results 
for band_name, models in model_results.items():
    freq_range = freq_band_ranges[band_names.index(band_name)]
    for model_name, model in models.items():
        if model:
            params = model.params
            pvalues = model.pvalues
            aic = model.aic
            bic = model.bic
            std_errs = model.bse  
            tvalues = model.tvalues  
            conf_int = model.conf_int()  
            
            for effect, coefficient in params.items():
                t_value = tvalues.get(effect, None)
                std_err = std_errs.get(effect, None)
                lower_bound, upper_bound = conf_int.loc[effect].values
                
                results_list.append({
                    "frequency band": band_name,
                    #"Frequency Range": freq_range,
                    "model": model_name,
                    "effect": effect,
                    "coef": custom_round(coefficient),  
                    "SE": custom_round(std_err),        
                    "p": custom_round(pvalues.get(effect, None)),  
                    "t": custom_round(t_value),        
                    "[0.025": custom_round(lower_bound),  
                    "0.975]": custom_round(upper_bound), 
                    #"AIC": custom_round(aic),           
                    #"BIC": custom_round(bic),          
                })
                

results_df = pd.DataFrame(results_list)

condition_pvalues = []
for band_name in band_names:
    print(band_name)
    for model_name, model in model_results[band_name].items():
        if model:
            pvalues = model.pvalues
            if 'C(Condition)[T.perceive_music_produced]' in pvalues.index:
                condition_pvalues.append(pvalues['C(Condition)[T.perceive_music_produced]'])

_, corrected_condition_pvalues = fdrcorrection(condition_pvalues)

print("Corrected p-values for 'Condition':")
for corrected_p in corrected_condition_pvalues:
    print(custom_round(corrected_p))

In [ ]:
# Model "Score ~ C(Condition) * C(Region)"
freq_band_index = [[0, 13], [13, 20], [20, 24], [24, 33], [33, 42], [42, 49]]
band_names = ["Delta", "Theta", "Alpha", "Beta", "Low Gamma", "High Gamma"]
freq_band_ranges = [(frequencies[fmin], frequencies[fmax]) for fmin, fmax in freq_band_index]

model_results = {band_name: {} for band_name in band_names}

# Loop over frequency bands
for (fmin, fmax), band_name in zip(freq_band_ranges, band_names):
    if band_name == "High Gamma":
        data_band = data_music[(data_music['Frequency'] >= fmin) & (data_music['Frequency'] <= fmax)]
    else:
        data_band = data_music[(data_music['Frequency'] >= fmin) & (data_music['Frequency'] < fmax)]
    
    data_band_avg = data_band.groupby(['Day', 'Condition', 'Channel', 'Epoch'], as_index=False).agg({
        'Score': 'mean'  
    })
    
    data_band_avg['Region'] = data_band_avg['Channel'].map(channel_to_region).fillna('Unknown')  
    
    # Model 
    model_condition_region = smf.mixedlm(
        formula="Score ~ C(Condition) * C(Region)",  
        data=data_band_avg,
        groups=data_band_avg["Channel"]  
    ).fit(reml=False)
    
    model_results[band_name]["Model Condition * Region"] = model_condition_region

results_list = []

# Loop through the regression results 
for band_name, models in model_results.items():
    freq_range = freq_band_ranges[band_names.index(band_name)]
    for model_name, model in models.items():
        if model:
            params = model.params
            pvalues = model.pvalues
            aic = model.aic
            bic = model.bic
            std_errs = model.bse  
            tvalues = model.tvalues  
            conf_int = model.conf_int()  
            
            for effect, coefficient in params.items():
                t_value = tvalues.get(effect, None)
                std_err = std_errs.get(effect, None)
                lower_bound, upper_bound = conf_int.loc[effect].values
                
                results_list.append({
                    "frequency band": band_name,
                    "model": model_name,
                    "effect": effect,
                    "coef": coefficient,
                    "SE": std_err,
                    "p": pvalues.get(effect, None),
                    "t": t_value,
                    "[0.025": lower_bound,
                    "0.975]": upper_bound,
                    "AIC": aic,
                    "BIC": bic,
                })

results_df = pd.DataFrame(results_list)

### Model for speech 

In [ ]:
channel_index = np.r_[42:50, 89:96]
fmin = 1
fmax = 180
frequencies = np.logspace(np.log10(fmin), np.log10(fmax), num = 50, base = 10)  # log10 freq scale 
epochs=np.r_[0:100]
data = []
for day, morlet_results in zip(['Day1', 'Day2'], [morlet_results_day1, morlet_results_day2]):
    for condition in ['produce_speech', 'perceive_speech_produced']:
        for pick, channel_idx in zip(picks_H, channel_index):
            for freq_idx, freq in enumerate(frequencies):
                for epoch_idx in epochs:
                    result = morlet_results[condition][epoch_idx, channel_idx, freq_idx]
                    data.append({
                        'Condition': condition, 
                        'Day': day, 
                        'Channel': pick, 
                        'Frequency': freq, 
                        'Epoch': epoch_idx, 
                        'Score': result
                    })

data_speech = pd.DataFrame(data)

data_speech['Condition'] = pd.Categorical(data_speech['Condition'], 
                                          categories=['produce_speech', 'perceive_speech_produced'],
                                          ordered=True)

data_speech['Region'] = data_speech['Channel'].map(channel_to_region)

In [ ]:
# Compare Models 
freq_band_index = [[0, 13], [13, 20], [20, 24], [24, 33], [33, 42], [42, 49]]
band_names = ["Delta", "Theta", "Alpha", "Beta", "Low Gamma", "High Gamma"]
freq_band_ranges = [(frequencies[fmin], frequencies[fmax]) for fmin, fmax in freq_band_index]

model_results = {}
results_list = []

# Loop over frequency bands
for (fmin, fmax), band_name in zip(freq_band_ranges, band_names):
    if band_name == "High Gamma":
        data_band = data_speech[(data_speech['Frequency'] >= fmin) & (data_speech['Frequency'] <= fmax)]
    else:
        data_band = data_speech[(data_speech['Frequency'] >= fmin) & (data_speech['Frequency'] < fmax)]
    
    data_band_avg = data_band.groupby(['Day', 'Condition', 'Channel', 'Epoch'], as_index=False).agg({
        'Score': 'mean'  #
    })
    
    data_band_avg['Region'] = data_band_avg['Channel'].map(channel_to_region)

    # Model 1: Fixed effect of Condition 
    model1 = smf.mixedlm(
        formula="Score ~ C(Condition)",  
        data=data_band_avg,
        groups=data_band_avg["Channel"]  
    ).fit(reml=False)
    model_results[band_name] = {"Model Condition": model1}


    # Model 2: Fixed effects of Condition and Region
    model2 = smf.mixedlm(
        formula="Score ~ C(Condition) + C(Region)",  
        data=data_band_avg,
        groups=data_band_avg["Channel"]  
    ).fit(reml=False)

    model_results[band_name]["Model Condition + Region"] = model2

    # Likelihood Ratio Test statistic
    ll_model_1 = model1.llf  
    ll_model_2 = model2.llf  
    num_params_model_1 = len(model1.params)
    num_params_model_2 = len(model2.params)
    
    lr_statistic = 2 * (ll_model_2 - ll_model_1)
    df_diff = num_params_model_2 - num_params_model_1
    p_value = 1 - stats.chi2.cdf(lr_statistic, df_diff)

    print(f"\n{band_name}: Likelihood Ratio Test between Model 1 and Model 2: Statistic = {lr_statistic}, p-value = {p_value}")

    results_list.append({
        "Stimuli": "speech", 
        "Frequency Band": band_name,
        "Model1": "Condition",
        "Model2": "Condition + Region",
        "AIC_Model1": model1.aic,
        "BIC_Model1": model1.bic,
        "AIC_Model2": model2.aic,
        "BIC_Model2": model2.bic,
        "Log-Likelihood_Model1": ll_model_1,
        "Log-Likelihood_Model1": ll_model_2,
        "LR Statistic": lr_statistic,
        "p-value": p_value
    })

df_results = pd.DataFrame(results_list)

In [ ]:
# Compare Model
freq_band_index = [[0, 13], [13, 20], [20, 24], [24, 33], [33, 42], [42, 49]]
band_names = ["Delta", "Theta", "Alpha", "Beta", "Low Gamma", "High Gamma"]
freq_band_ranges = [(frequencies[fmin], frequencies[fmax]) for fmin, fmax in freq_band_index]

model_results = {}
results_list = []

# Loop over frequency bands
for (fmin, fmax), band_name in zip(freq_band_ranges, band_names):
    if band_name == "High Gamma":
        data_band = data_speech[(data_speech['Frequency'] >= fmin) & (data_speech['Frequency'] <= fmax)]
    else:
        data_band = data_speech[(data_speech['Frequency'] >= fmin) & (data_speech['Frequency'] < fmax)]
    
    data_band_avg = data_band.groupby(['Day', 'Condition', 'Channel', 'Epoch'], as_index=False).agg({
        'Score': 'mean'  #
    })
    
    data_band_avg['Region'] = data_band_avg['Channel'].map(channel_to_region)

    # Model 1: Fixed effect of Condition + Region
    model1 = smf.mixedlm(
        formula="Score ~ C(Condition) + C(Region)",  
        data=data_band_avg,
        groups=data_band_avg["Channel"]  
    ).fit(reml=False)
    model_results[band_name] = {"Model Condition + Region": model1}


    # Model 2: Fixed effects of Condition * Region
    model2 = smf.mixedlm(
        formula="Score ~ C(Condition) * C(Region)",  
        data=data_band_avg,
        groups=data_band_avg["Channel"]  
    ).fit(reml=False)

    model_results[band_name]["Model Condition * Region"] = model2

    # Likelihood Ratio Test statistic
    ll_model_1 = model1.llf  
    ll_model_2 = model2.llf  
    num_params_model_1 = len(model1.params)
    num_params_model_2 = len(model2.params)
    
    lr_statistic = 2 * (ll_model_2 - ll_model_1)
    df_diff = num_params_model_2 - num_params_model_1
    p_value = 1 - stats.chi2.cdf(lr_statistic, df_diff)

    print(f"\n{band_name}: Likelihood Ratio Test between Model 1 and Model 2: Statistic = {lr_statistic}, p-value = {p_value}")

    results_list.append({
        "Stimuli": "speech", 
        "Frequency Band": band_name,
        "Model1": "Condition + Region",
        "Model2": "Condition * Region",
        "AIC_Model1": model1.aic,
        "BIC_Model1": model1.bic,
        "AIC_Model2": model2.aic,
        "BIC_Model2": model2.bic,
        "Log-Likelihood_Model1": ll_model_1,
        "Log-Likelihood_Model1": ll_model_2,
        "LR Statistic": lr_statistic,
        "p-value": p_value
    })

df_results = pd.DataFrame(results_list)

In [ ]:
# Model "Score ~ C(Condition)"
freq_band_index = [[0, 13], [13, 20], [20, 24], [24, 33], [33, 42], [42, 49]]
band_names = ["Delta", "Theta", "Alpha", "Beta", "Low Gamma", "High Gamma"]
freq_band_ranges = [(frequencies[fmin], frequencies[fmax]) for fmin, fmax in freq_band_index]

model_results = {band_name: {} for band_name in band_names}

# Loop over frequency bands
for (fmin, fmax), band_name in zip(freq_band_ranges, band_names):
    if band_name == "High Gamma":
        data_band = data_speech[(data_speech['Frequency'] >= fmin) & (data_speech['Frequency'] <= fmax)]
    else:
        data_band = data_speech[(data_speech['Frequency'] >= fmin) & (data_speech['Frequency'] < fmax)]
    
    data_band_avg = data_band.groupby(['Day', 'Condition', 'Channel', 'Epoch'], as_index=False).agg({
        'Score': 'mean'  
    })
    
    data_band_avg['Region'] = data_band_avg['Channel'].map(channel_to_region).fillna('Unknown') 

    # Model 
    model_condition = smf.mixedlm(
        formula="Score ~ C(Condition)", 
        data=data_band_avg,                            
        groups=data_band_avg["Channel"]     
    ).fit(reml=False)
    model_results[band_name] = {"Model Condition": model_condition}


results_list = []

# Loop through the regression results 
for band_name, models in model_results.items():
    freq_range = freq_band_ranges[band_names.index(band_name)]
    for model_name, model in models.items():
        if model:
            params = model.params
            pvalues = model.pvalues
            aic = model.aic
            bic = model.bic
            std_errs = model.bse  
            tvalues = model.tvalues  
            conf_int = model.conf_int()  
            
            for effect, coefficient in params.items():
                t_value = tvalues.get(effect, None)
                std_err = std_errs.get(effect, None)
                lower_bound, upper_bound = conf_int.loc[effect].values
                
                results_list.append({
                    "frequency band": band_name,
                    "model": model_name,
                    "effect": effect,
                    "coef": custom_round(coefficient),  
                    "SE": custom_round(std_err),        
                    "p": custom_round(pvalues.get(effect, None)),  
                    "t": custom_round(t_value),        
                    "[0.025": custom_round(lower_bound),  
                    "0.975]": custom_round(upper_bound), 
                    #"AIC": custom_round(aic),           
                    #"BIC": custom_round(bic),          
                })
                

results_df = pd.DataFrame(results_list)

condition_pvalues = []

for band_name in band_names:
    print(band_name)
    for model_name, model in model_results[band_name].items():
        if model:
            pvalues = model.pvalues
            if 'C(Condition)[T.perceive_speech_produced]' in pvalues.index:
                condition_pvalues.append(pvalues['C(Condition)[T.perceive_speech_produced]'])

_, corrected_condition_pvalues = fdrcorrection(condition_pvalues)

print("Corrected p-values for 'Condition':")
for corrected_p in corrected_condition_pvalues:
    print(custom_round(corrected_p))

In [ ]:
# Model "Score ~ C(Condition) * C(Region)"
freq_band_index = [[0, 13], [13, 20], [20, 24], [24, 33], [33, 42], [42, 49]]
band_names = ["Delta", "Theta", "Alpha", "Beta", "Low Gamma", "High Gamma"]
freq_band_ranges = [(frequencies[fmin], frequencies[fmax]) for fmin, fmax in freq_band_index]

model_results = {band_name: {} for band_name in band_names}

# Loop over frequency bands
for (fmin, fmax), band_name in zip(freq_band_ranges, band_names):
    if band_name == "High Gamma":
        data_band = data_speech[(data_speech['Frequency'] >= fmin) & (data_speech['Frequency'] <= fmax)]
    else:
        data_band = data_speech[(data_speech['Frequency'] >= fmin) & (data_speech['Frequency'] < fmax)]
    
    data_band_avg = data_band.groupby(['Day', 'Condition', 'Channel', 'Epoch'], as_index=False).agg({
        'Score': 'mean'  
    })
    
    data_band_avg['Region'] = data_band_avg['Channel'].map(channel_to_region).fillna('Unknown')  
    
    # Model 
    model_condition_region = smf.mixedlm(
        formula="Score ~ C(Condition) * C(Region)",  
        data=data_band_avg,
        groups=data_band_avg["Channel"] 
    ).fit(reml=False)
    
    model_results[band_name]["Model Condition * Region"] = model_condition_region


results_list = []

# Loop through the regression results 
for band_name, models in model_results.items():
    freq_range = freq_band_ranges[band_names.index(band_name)]
    for model_name, model in models.items():
        if model:
            params = model.params
            pvalues = model.pvalues
            aic = model.aic
            bic = model.bic
            std_errs = model.bse  
            tvalues = model.tvalues  
            conf_int = model.conf_int()  
            
            for effect, coefficient in params.items():
                t_value = tvalues.get(effect, None)
                std_err = std_errs.get(effect, None)
                lower_bound, upper_bound = conf_int.loc[effect].values
                
                results_list.append({
                    "frequency band": band_name,
                    "model": model_name,
                    "effect": effect,
                    "coef": coefficient,
                    "SE": std_err,
                    "p": pvalues.get(effect, None),
                    "t": t_value,
                    "[0.025": lower_bound,
                    "0.975]": upper_bound,
                    "AIC": aic,
                    "BIC": bic,
                })

results_df = pd.DataFrame(results_list)

## Speech and Music Repetition: First Hearing - Second Hearing 

In [ ]:
channel_index = np.r_[42:50, 89:96]
brain_label = ["Right Primary \n Auditory Cortex", "Right Associative \n Auditory Cortex", "Left Primary \n Auditory Cortex", "Left Associative \n Auditory Cortex"]
channel_position = [[0,5], [6,7], [8,11], [11,len(picks_H)]]

for n, contrast in enumerate(contrasts_control): 
    data_firsthearing = morlet_results_control[contrast[0]][:,channel_index,:]
    data_secondhearing = morlet_results_control[contrast[1]][:,channel_index,:]
    
    ratio = np.mean(((data_firsthearing - data_secondhearing ) / data_secondhearing * 100), axis=0)

    t, p = stats.ttest_rel(data_firsthearing, data_secondhearing)
    p_corrected = multipletests(p.flatten(), method='fdr_bh')[1].reshape(p.shape)
    t[p_corrected > 0.05] = 0
    
    fig, ax = plt.subplots(figsize=(12, 8), ncols=1)

    im1 = ax.imshow(t, aspect='auto', origin='lower', vmin=-10, vmax=10, interpolation='none',  cmap='seismic')
    ax.set_xlabel('Frequencies', size=16)
    ax.set_ylabel('Channels', size=16)
    ax.set_yticks(np.arange(t.shape[0]))
    ax.set_yticklabels(picks_H, size=12)
    ax.set_xticks(np.arange(len(frequencies)))
    ax.set_xticklabels((np.around(frequencies,1)), rotation=90, size=12)
    ax.axhline(7.5, color="black", lw=3)
    for channel_pos_index in [[0,5], [5,7], [7,11]]: 
        ax.axhline(channel_pos_index[1] + 0.5, color="black", lw=1.5)
    for band_index in [[0,13], [13,20], [20,24], [24,33], [33,42]]:
        ax.axvline(band_index[1] - 0.5, color="black", lw=1)

    ax2 = ax.twiny()  
    ax2.set_xlim(ax.get_xlim()) 
    band_midpoints = [(index[0] + index[1]) / 2 for index in freq_band_index]
    ax2.set_xticks(band_midpoints)  
    ax2.set_xticklabels(band_names, rotation=0, ha='center', size=12)  
    ax2.tick_params(axis='x', pad=10)  
    fig.subplots_adjust(bottom=0.2, top=0.85)

    ax3 = ax.twinx()  
    ax3.set_ylim(ax.get_ylim()) 
    channel_midpoints = [(index[0] + index[1]) / 2 for index in channel_position]
    ax3.set_yticks(channel_midpoints)  
    ax3.set_yticklabels(brain_label, size=12, ha='left')  

    colorbar = fig.colorbar(im1, ax=ax, fraction=0.05, pad=0.17)
    colorbar.set_label('t-statistics', size=14) 

    contrast_name = contrast[0] + "-" + contrast[1]
    plt.show()


## Linear mixed effect models

### Model for music 

In [ ]:
channel_index = np.r_[42:50, 89:96]
frequencies = np.logspace(np.log10(fmin), np.log10(fmax), num=50, base=10)
epochs = np.r_[0:100]

data = []

for condition in ['perceive_music_new', 'perceive_music_newrepetition']:
    for pick, channel_idx in zip(picks_H, channel_index):
        for freq_idx, freq in enumerate(frequencies):
            for epoch_idx in epochs:
                result = morlet_results_control[condition][epoch_idx, channel_idx, freq_idx]
                data.append({
                    'Condition': condition, 
                    'Channel': pick, 
                    'Frequency': freq, 
                    'Epoch': epoch_idx, 
                    'Score': result
                })

data_music = pd.DataFrame(data)

data_music['Condition'] = pd.Categorical(data_music['Condition'], 
                                          categories=['perceive_music_new', 'perceive_music_newrepetition'],
                                          ordered=True)


In [ ]:
# Model "Score ~ C(Condition)"
freq_band_index = [[0, 13], [13, 20], [20, 24], [24, 33], [33, 42], [42, 49]]
band_names = ["Delta", "Theta", "Alpha", "Beta", "Low Gamma", "High Gamma"]
freq_band_ranges = [(frequencies[fmin], frequencies[fmax]) for fmin, fmax in freq_band_index]

model_results = {band_name: {} for band_name in band_names}

# Loop over frequency bands
for (fmin, fmax), band_name in zip(freq_band_ranges, band_names):
    if band_name == "High Gamma":
        data_band = data_music[(data_music['Frequency'] >= fmin) & (data_music['Frequency'] <= fmax)]
    else:
        data_band = data_music[(data_music['Frequency'] >= fmin) & (data_music['Frequency'] < fmax)]
    
    data_band_avg = data_band.groupby(['Condition', 'Channel', 'Epoch'], as_index=False).agg({
        'Score': 'mean'  
    })
    
    data_band_avg['Region'] = data_band_avg['Channel'].map(channel_to_region).fillna('Unknown') 

    # Model
    model_condition = smf.mixedlm(
        formula="Score ~ C(Condition)", 
        data=data_band_avg,                            
        groups=data_band_avg["Channel"]     
    ).fit(reml=False)
    model_results[band_name] = {"Model Condition": model_condition}


results_list = []

# Loop through the regression results 
for band_name, models in model_results.items():
    for model_name, model in models.items():
        if model:
            params = model.params
            pvalues = model.pvalues
            aic = model.aic
            bic = model.bic
            std_errs = model.bse  
            tvalues = model.tvalues  
            conf_int = model.conf_int()  
            
            for effect, coefficient in params.items():
                t_value = tvalues.get(effect, None)
                std_err = std_errs.get(effect, None)
                lower_bound, upper_bound = conf_int.loc[effect].values
                
                results_list.append({
                    "frequency band": band_name,
                    "model": model_name,
                    "effect": effect,
                    "coef": custom_round(coefficient),  
                    "SE": custom_round(std_err),        
                    "p": custom_round(pvalues.get(effect, None)),  
                    "t": custom_round(t_value),        
                    "[0.025": custom_round(lower_bound),  
                    "0.975]": custom_round(upper_bound), 
                    #"AIC": custom_round(aic),           
                    #"BIC": custom_round(bic),          
                })

results_df = pd.DataFrame(results_list)

condition_pvalues = []
for band_name in band_names:
    print(band_name)
    for model_name, model in model_results[band_name].items():
        if model:
            pvalues = model.pvalues
            if 'C(Condition)[T.perceive_music_newrepetition]' in pvalues.index:
                condition_pvalues.append(pvalues['C(Condition)[T.perceive_music_newrepetition]'])

_, corrected_condition_pvalues = fdrcorrection(condition_pvalues)

print("Corrected p-values for 'Condition':")
for corrected_p in corrected_condition_pvalues:
    print(custom_round(corrected_p))

#### Model for speech 

In [ ]:
channel_index = np.r_[42:50, 89:96]
frequencies = np.logspace(np.log10(fmin), np.log10(fmax), num=50, base=10)
epochs = np.r_[0:100]

data = []

for condition in ['perceive_speech_new', 'perceive_speech_newrepetition']:
    for pick, channel_idx in zip(picks_H, channel_index):
        for freq_idx, freq in enumerate(frequencies):
            for epoch_idx in epochs:
                result = morlet_results_control[condition][epoch_idx, channel_idx, freq_idx]
                data.append({
                    'Condition': condition, 
                    'Channel': pick, 
                    'Frequency': freq, 
                    'Epoch': epoch_idx, 
                    'Score': result
                })

data_speech = pd.DataFrame(data)

data_speech['Condition'] = pd.Categorical(data_speech['Condition'], 
                                          categories=['perceive_speech_new', 'perceive_speech_newrepetition'],
                                          ordered=True)

In [ ]:
# Model "Score ~ C(Condition)"
freq_band_index = [[0, 13], [13, 20], [20, 24], [24, 33], [33, 42], [42, 49]]
band_names = ["Delta", "Theta", "Alpha", "Beta", "Low Gamma", "High Gamma"]
freq_band_ranges = [(frequencies[fmin], frequencies[fmax]) for fmin, fmax in freq_band_index]

model_results = {band_name: {} for band_name in band_names}

# Loop over frequency bands
for (fmin, fmax), band_name in zip(freq_band_ranges, band_names):
    if band_name == "High Gamma":
        data_band = data_speech[(data_speech['Frequency'] >= fmin) & (data_speech['Frequency'] <= fmax)]
    else:
        data_band = data_speech[(data_speech['Frequency'] >= fmin) & (data_speech['Frequency'] < fmax)]
    
    data_band_avg = data_band.groupby(['Condition', 'Channel', 'Epoch'], as_index=False).agg({
        'Score': 'mean'  
    })
    
    data_band_avg['Region'] = data_band_avg['Channel'].map(channel_to_region).fillna('Unknown') 

    # Model 
    model_condition = smf.mixedlm(
        formula="Score ~ C(Condition)",  
        data=data_band_avg,                            
        groups=data_band_avg["Channel"]     
    ).fit(reml=False)
    model_results[band_name] = {"Model Condition": model_condition}


results_list = []

# Loop through the regression results 
for band_name, models in model_results.items():
    for model_name, model in models.items():
        if model:
            params = model.params
            pvalues = model.pvalues
            aic = model.aic
            bic = model.bic
            std_errs = model.bse  
            tvalues = model.tvalues  
            conf_int = model.conf_int()  
            
            for effect, coefficient in params.items():
                t_value = tvalues.get(effect, None)
                std_err = std_errs.get(effect, None)
                lower_bound, upper_bound = conf_int.loc[effect].values
                
                results_list.append({
                    "frequency band": band_name,
                    "model": model_name,
                    "effect": effect,
                    "coef": custom_round(coefficient),  
                    "SE": custom_round(std_err),        
                    "p": custom_round(pvalues.get(effect, None)),  
                    "t": custom_round(t_value),        
                    "[0.025": custom_round(lower_bound),  
                    "0.975]": custom_round(upper_bound), 
                    #"AIC": custom_round(aic),           
                    #"BIC": custom_round(bic),          
                })

results_df = pd.DataFrame(results_list)

condition_pvalues = []
for band_name in band_names:
    print(band_name)
    for model_name, model in model_results[band_name].items():
        if model:
            pvalues = model.pvalues            
            if 'C(Condition)[T.perceive_speech_newrepetition]' in pvalues.index:
                condition_pvalues.append(pvalues['C(Condition)[T.perceive_speech_newrepetition]'])

print(condition_pvalues)
_, corrected_condition_pvalues = fdrcorrection(condition_pvalues)

print("Corrected p-values for 'Condition':")
for corrected_p in corrected_condition_pvalues:
    print(custom_round(corrected_p))